<a href="https://colab.research.google.com/github/teobenko99/PRACTICA/blob/main/03_ING_FEAT_TP_Final_AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Carga de Librerias

In [ ]:
#Cargas
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt



# Opcional, pero recomendado para la imputación
from sklearn.impute import SimpleImputer

# Definición de la función
def load_data():
    candidates = [
        Path('/content/dataset_limpio_ok.csv'),   # Colab local
        Path('data/raw/dataset_limpio_ok.csv'),   # Proyecto local
        Path('dataset_limpio_ok.csv'),            # Carpeta actual
    ]
    for p in candidates:
        if p.exists():
            df = pd.read_csv(p)
            print("=== Dataset cargado ===")
            print(f"Archivo: {p} | Filas={df.shape[0]} | Columnas={df.shape[1]}")
            display(df.head(3))
            return df
    raise FileNotFoundError('No se encontró dataset_limpio_ok.csv en rutas conocidas.')

#ver DS cargado
df = load_data()  # Llama a la función y guarda el DataFrame en 'df'

=== Dataset cargado ===
Archivo: /content/dataset_limpio_ok.csv | Filas=235 | Columnas=36


,profesionales,edad,motivo_de_consulta,medio_por_el_que_ingresa,genero,nacionalidad,barrio,municipio,localidad,estado_civil,...,simbolica,ambiental,politica,digital,cant_tipos_violencias_por_persona,denuncio,medidas_de_proteccion,fecha_fin_de_vigencia,personas_a_cargo,red_vincular
0,fanny___dana,28.0,violencia,desconocido,mujer,argentina,bordeu,bahia_blanca,bahia_blanca,soltera,...,0,0,0,0,2.0,no,no,NaN,no,parientes_convivientes
1,agus__analé,43.0,violencia,desconocido,mujer,argentina,pacifico,bahia_blanca,bahia_blanca,desconocido,...,0,1,0,0,4.0,si,prohibición_de_acercamiento,NaN,hija_hijo,parientes_no_convivientes
2,fanny__majo,53.0,violencia,desconocido,mujer,argentina,pampa_central,bahia_blanca,bahia_blanca,soltera,...,0,0,0,0,2.0,no,no,NaN,no,parientes_no_convivientes


In [ ]:
import unicodedata
import re

# Universidad Nacional de la Matanza

Especialización en Ciencia de Datos

Materia: APRENDIZAJE AUTOMÁTICO

GRUPO : Benko Teo, Cura Diego, Riganti Valentina, Sanjuan Oriana.

# **Etapa: Ingeniería de Características**

### Análisis de variables de consulta (qué profesional se asignó, cuál es el motivo de la consulta y por qué canal llega):

Las variables profesionales y motivo_de_consulta, deben ser eliminadas por razones estratégicas (alta cardinalidad y bajo valor predictivo, respectivamente):

1. Decisión: Eliminar 'profesionales' por Cardinalidad Extrema (no se puede usar OHE).
2. Decisión: Eliminar 'motivo_de_consulta' por Sesgo/Baja Variabilidad (casi 86% es 'violencia').

In [ ]:
cols_to_engineer = ['profesionales', 'motivo_de_consulta']

df = df.drop(columns=cols_to_engineer, errors='ignore')

# Verificación de la Ingeniería
print("--- Ingeniería de Características: Eliminación de Variables ---")
print(f"Columnas eliminadas por estrategia: {cols_to_engineer}")
print(f"Total de columnas restantes: {df.shape[1]}")

--- Ingeniería de Características: Eliminación de Variables ---
Columnas eliminadas por estrategia: ['profesionales', 'motivo_de_consulta']
Total de columnas restantes: 34


La variable 'medio_por_el_que_ingresa' tiene alta cardinalidad (10 categorías) pero muchas de baja frecuencia (menos del $3\%$ de los datos).

Decisión: agrupar funcionalmente las categorías pequeñas para mejorar la estabilidad del modelo sin perder información esencial sobre el canal de consulta.

La agrupación de los canales originales resulta en:

| Categoría Funcional | Categorías Originales | Observación |
| --- | --- | --- |
| CANAL_JUDICIAL | notificación_judicial | Representa el canal más formal/obligatorio de ingreso |
| CANAL_ESPONTANEO | espontánea | Alto valor predictivo (iniciativa de la víctima) |
| CANAL_TERCERO | otra | Captura las derivaciones no institucionales directas |
| CANAL_ASISTENCIAL | "turno_programado, comisaría, unidad_sanitaria, organización_social..., institución_educativa, hospital" | Agrupamiento Estratégico: la víctima fue derivada o asistida por otra entidad (salud, policía, social) |
| DESCONOCIDO | desconocido | Preserva la información de ausencia de dato |

In [ ]:
columna = 'medio_por_el_que_ingresa'
nuevo_nombre = 'canal_ingreso_agrupado'

# Aplicación Agrupamiento Funcional
df[nuevo_nombre] = np.select(
    [
        # 1. CANAL JUDICIAL
        df[columna].str.contains('notificación_judicial', na=False),

        # 2. CANAL ESPONTANEO
        df[columna] == 'espontánea',

        # 3. CANAL TERCERO
        df[columna] == 'otra',

        # 4. CANAL ASISTENCIAL (Agrupa todos los de baja frecuencia/instituciones)
        df[columna].isin(['turno_programado', 'comisaría', 'unidad_sanitaria', 'organización_social_institución_comunitaria', 'institución_educativa', 'hospital']),

        # 5. DESCONOCIDO
        df[columna] == 'desconocido'
    ],
    [
        'CANAL_JUDICIAL',
        'CANAL_ESPONTANEO',
        'CANAL_TERCERO',
        'CANAL_ASISTENCIAL',
        'DESCONOCIDO'
    ],
    default='OTROS_A_REVISAR' # Categoría de seguridad para cualquier valor no mapeado
)

# Codificación OHE
df = pd.get_dummies(df, columns=[nuevo_nombre], prefix='INGR', drop_first=False)

# Eliminar la columna original (ya transformada)
df = df.drop(columns=[columna])

# Verificación
print("\n--- Verificación del Agrupamiento de 'medio_por_el_que_ingresa' ---")
print(df[[col for col in df.columns if col.startswith('INGR_')]].sum().to_frame(name='Conteo Final').to_markdown())


--- Verificación del Agrupamiento de 'medio_por_el_que_ingresa' ---
|                        |   Conteo Final |
|:-----------------------|---------------:|
| INGR_CANAL_ASISTENCIAL |             28 |
| INGR_CANAL_ESPONTANEO  |             50 |
| INGR_CANAL_JUDICIAL    |             83 |
| INGR_CANAL_TERCERO     |             26 |
| INGR_DESCONOCIDO       |             48 |


### Análisis de variables que describen la ubicación de la persona que se comunica:

Las variables 'municipio' y 'localidad' aportan información redundante y tienen poco valor predictivo, dado que mas de un 95% de los casos ocurren en Bahía Blanca.

Decisión: Eliminar 'municipio'. Después de la limpieza, el $98.3\%$ de los casos son bahía_blanca. Es una feature casi constante y no aporta valor predictivo al modelo.

Decisión: Eliminar 'localidad'. Después de la limpieza, el $97\%$ de los casos son bahía_blanca. Es una feature casi constante y es redundante con municipio (ya eliminado).

In [ ]:
df = df.drop(columns=['municipio', 'localidad'], errors='ignore')

La variable 'nacionalidad' está fuertemente sesgada hacia argentina ($93.19\%$). Para convertirla en una feature útil, se aplicará un agrupamiento binario simple. Así, se evitan sesgos de la clase mayoritaria y se conserva la información sobre otras nacionalidades que no sean argentina.



In [ ]:
col_nac = 'nacionalidad'
df['nacionalidad_agrupada'] = np.where(
    df[col_nac] == 'argentina',
    'ARGENTINA',
    'NO_ARGENTINA'
)

print("--- Verificación de la Ingeniería de Ubicación ---")
print("\nConteo de Nacionalidad Agrupada:")
print(df[[col for col in df.columns if col.startswith('NAC_')]].sum().to_frame(name='Conteo').to_markdown())

--- Verificación de la Ingeniería de Ubicación ---

Conteo de Nacionalidad Agrupada:
| Conteo   |
|----------|


In [ ]:
df = df.drop(columns=[col_nac], errors='ignore')

Para la variable 'barrio', se va a realizar una zonificación con el fin de reducir las casi 100 categorías a 7 zonas. De esta manera, se preserva el conocimiento geográfico.

In [ ]:
# Transformación de barrios (Zonificación)

# Listas de clasificación normalizadas (minúsculas, sin tildes/símbolos)

zona_centro_norm = [
    'centro', 'microcentro', 'naposta', 'pacifico', 'b pacifico', 'pedro pico', 'ricchieri'
]

zona_norte_norm = [
    'agua blanca', 'altos de bahia', 'altos de la carrindanga', 'jardin del este', 'la falda',
    'los alamos', 'millamapu', 'molina campos', 'palihue', 'b patagonia', 'patagonia norte',
    'parque del sol', 'parque norte', 'parque patagonia', 'parque sesquicentenario',
    'portal del este', 'san agustin', 'san ignacio', 'santa margarita', 'solares norte', 'universitario'
]

zona_villas_norm = [
    '1ro de mayo', '5 de abril', '27 de junio', 'bap', 'comercial', 'evita', 'barrio evita',
    'rosendo lopez', 'sutiaga', 'uom', 'upcn', 'villa amaducci', 'villa belgrano',
    'villa cerrito', 'villa floresta', 'villa libre', 'villa mitre', 'villa gral mitre',
    'villa rosario', 'villa rosas', 'villa serra', 'villa soldati'
]

zona_noroeste_norm = [
    'aldea romana', 'altos del sur', 'ate', 'ate 1', 'ate 2', 'ate 3', 'ate 4', 'ate 5',
    'avellaneda', 'bancario', 'bellavista', 'cgt', 'campana del desierto', 'cooperacion 2',
    'coronel maldonado', 'don bosco', 'don ramiro', 'el bosque', 'el matadero', 'el nacional',
    'el prado', 'empl de comercio', 'floresta', 'fonavi', '9 de noviembre', 'la canada',
    'las canitas', 'latino', 'loma alta', 'loma paraguaya', 'los chanares', 'lujan',
    'luz y fuerza', 'malvinas argentinas', 'mapuche', 'mataderos', 'nor oeste', 'nueva central',
    'palos verdes', 'pampa central', 'piedra buena', 'polo', 'prensa', 'suem',
    'san cayetano', 'san francisco', 'san jorge', 'san martin', 'san miguel', 'santa catalina',
    'santa teresita', 'spurr', 'stella maris', 'thompson', 'viajantes del sur', 'villa alegre',
    'villa delfina', 'villa don bosco', 'villa duprat', 'villa elena', 'villa esperan',
    'villa gloria', 'villa hipodromo', 'villa irupe', 'villa italia', 'villa loreto',
    'villa moresino', 'villa muniz', 'villa nocito', 'villa nueva', 'villa ressia',
    'villa rica', 'villa roma', 'villa sanchez', 'vista alegre'
]

zona_exteriores_norm = [
    '17 de mayo', '26 de septiembre', 'aeropuerto', 'villa aeropuerto', 'amef',
    'boulevard white', 'cnel falcon', 'essa', 'essa 2', 'espora', 'general cerri',
    'grunbein', 'harding green', 'villa harding green', 'ingeniero white', 'kilometro 5',
    'obrero', 'parque industrial', 'pescadores', 'petroquimico', 'playa serena', 'saladero',
    'supe', 'villa bordeu', 'villa gral arias'
]

# Función de clasificación principal (Lógica modificada)

def clasificar_zona(barrio):
    """
    Toma el barrio PRE-LIMPIADO y lo adapta al formato de las listas de zonificación (con espacios).
    """
    # Adaptación del input: Pasa a minúsculas y convierte GUIONES BAJOS a ESPACIOS
    barrio_norm = str(barrio).strip().lower().replace('_', ' ')

    # 1. Chequear nulos o 'sin datos' explícito
    # (Si la normalización da un string vacío, era nulo o 'nan')
    if barrio_norm == 'desconocida':
        return 'DESCONOCIDO' ## ---> modificado por la limpieza

    # 2. Chequear zonas principales (comparando el formato adaptado con las listas)
    # NOTA: Usamos barrio_norm directamente para la comparación, ya que ahora tiene espacios.

    if barrio_norm in zona_centro_norm:
        return 'CENTRO_Y_MACROCENTRO'
    if barrio_norm in zona_norte_norm:
        return 'NORTE'
    if barrio_norm in zona_villas_norm:
        return 'SUR'
    if barrio_norm in zona_noroeste_norm:
        return 'NOROESTE'
    if barrio_norm in zona_exteriores_norm:
        return 'INGENIERO_WHITE'

    # 3. Chequear casos parciales (ej. fonavi i, fonavi ii)
    #(siempre usando el formato adaptado con espacio)
    if 'fonavi' in barrio_norm:
        return 'NOROESTE'
    if 'ate' in barrio_norm:
        return 'NOROESTE'
    if 'harding green' in barrio_norm:
        return 'INGENIERO_WHITE'
    if 'bordeu' in barrio_norm:
        return 'NORTE'

    # 4. Si no coincidió con nada de lo anterior, es 'otros'
    return 'OTROS'

# LÍNEA DE APLICACIÓN

df['ZONA'] = df['barrio'].apply(clasificar_zona)

In [ ]:
print(df['ZONA'].value_counts())

ZONA
OTROS                   84
NOROESTE                57
SUR                     29
DESCONOCIDO             20
INGENIERO_WHITE         20
NORTE                   14
CENTRO_Y_MACROCENTRO    11
Name: count, dtype: int64


In [ ]:
# Eliminación de la columna original
df = df.drop(columns=['barrio'], errors='ignore')

In [ ]:
# CODIFICACIÓN OHE
ohe_targets = ['nacionalidad_agrupada', 'ZONA']
df = pd.get_dummies(df, columns=ohe_targets, prefix=['NAC', 'BA'], drop_first=False)


# 5. Verificación de la Ingeniería
print("--- Verificación de la Ingeniería de Ubicación ---")

print("\nConteo de Barrio Zonificado:")
print(df[[col for col in df.columns if col.startswith('BA_')]].sum().to_frame(name='Conteo').to_markdown())

--- Verificación de la Ingeniería de Ubicación ---

Conteo de Barrio Zonificado:
|                         |   Conteo |
|:------------------------|---------:|
| BA_CENTRO_Y_MACROCENTRO |       11 |
| BA_DESCONOCIDO          |       20 |
| BA_INGENIERO_WHITE      |       20 |
| BA_NOROESTE             |       57 |
| BA_NORTE                |       14 |
| BA_OTROS                |       84 |
| BA_SUR                  |       29 |


### Análisis de variables que describen características de la persona:

Se crea una variable binaria para identificar a los generos. Esto permite agrupar la clase minoritaria ('otro') y desconocida, en una categória conocida -> 'NO_MUJER'.

Así, se capturan las posibilidades que no sea mujer la persona que consulta.

In [ ]:
# INGENIERÍA DE 'genero' (Agrupamiento Binario)
col_genero = 'genero'
grouped_col_genero = 'genero_grouped'

# Aplicar la lógica de agrupamiento (Mujer vs. Todo lo demás)
df[grouped_col_genero] = np.where(
    # Asumimos que la columna ya está en minúsculas ('mujer') por la limpieza
    df[col_genero] == 'mujer',
    'MUJER',
    'NO_MUJER'
)

# Aplicar OHE a la nueva variable (solo se generan 2 columnas, 1 se elimina con drop_first=True, pero la dejaremos completa por ahora)
df = pd.get_dummies(df, columns=[grouped_col_genero], prefix='GEN', drop_first=False)


# VERIFICACIÓN
print("--- Verificación de la Ingeniería de Características (Edad) ---")
genero_cols = [col for col in df.columns if col.startswith('GEN_')]
print("\nConteo de Género Agrupado (GEN):")
print(df[genero_cols].sum().to_frame(name='Conteo').to_markdown())

--- Verificación de la Ingeniería de Características (Edad) ---

Conteo de Género Agrupado (GEN):
|              |   Conteo |
|:-------------|---------:|
| GEN_MUJER    |      228 |
| GEN_NO_MUJER |        7 |


In [ ]:
# Eliminar columna original
df = df.drop(columns=[col_genero], errors='ignore')

Se establecen rangos etarios para poder agrupar la variable numerica continua 'edad'.

Se utilizaron para la estabilidad de las categorías una división de rangos basada en los cuartiles Q1=29.5, Mediana=36, Q3=45, y la edad maxima=85.

In [ ]:
# INGENIERÍA DE 'edad' (Creación de Rangos Etarios)

col_edad = 'edad'
binned_col_edad = 'rango_etario'

# Definir los límites numéricos
bins = [17, 29.5, 36, 45, 85]
labels = ['17_a_29', '30_a_36', '37_a_45', '46_a_85'] # Usar guiones bajos para OHE

# Crear la variable binarizada (pd.cut)
df[binned_col_edad] = pd.cut(
    df[col_edad],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)

# Aplicar OHE a la nueva variable de rango
df = pd.get_dummies(df, columns=[binned_col_edad], prefix='EDAD', drop_first=False)


# VERIFICACIÓN
print("--- Verificación de la Ingeniería de Características (Edad) ---")

edad_cols = [col for col in df.columns if col.startswith('EDAD_')]
print("\nConteo de Rangos Etarios (EDAD):")
print(df[edad_cols].sum().to_frame(name='Conteo').to_markdown())

print(f"\nTipo de dato de 'edad' (debe ser float/int): {df['edad'].dtype}")


--- Verificación de la Ingeniería de Características (Edad) ---

Conteo de Rangos Etarios (EDAD):
|              |   Conteo |
|:-------------|---------:|
| EDAD_17_a_29 |       59 |
| EDAD_30_a_36 |       60 |
| EDAD_37_a_45 |       51 |
| EDAD_46_a_85 |       55 |

Tipo de dato de 'edad' (debe ser float/int): float64


In [ ]:
# 2.4. Eliminar la columna intermedia de rangos y la de edad
df = df.drop(columns=[binned_col_edad], errors='ignore')
df = df.drop(columns=['edad'], errors='ignore')

### Análisis de variables socioeconómicas y de estatus de la persona:

Para la variable 'estado civil' se decidió identificar dos categórias opuestas, que representan diferentes accionares posibles en las personas y sus vínculos.

Casada --> formalmente o por unión convivencial. Funcionalmente, ambos representan un vínculo formal/estable con obligaciones mutuas. Para el riesgo, el modelo solo necesita saber si el vínculo existe.

Separada/Divorciada --> Ambas representan el fin de un vínculo formal, lo que puede aumentar el riesgo de violencia post-ruptura.

In [ ]:
# --- 1. INGENIERÍA DE ESTADO CIVIL (EC) ---
col_ec = 'estado_civil'
# 1.1. Agrupamiento de Unión Convivencial en Casada
df[col_ec] = df[col_ec].replace('unión convivencial', 'casada')
# 1.2. Agrupamiento de Separada y Divorciada
df[col_ec] = df[col_ec].replace({'separada': 'separada_divorciada', 'divorciada': 'separada_divorciada'})
# 1.3. OHE aplicado a la columna limpia
df = pd.get_dummies(df, columns=[col_ec], prefix='EC', drop_first=False)
df = df.drop(columns=['estado_civil'], errors='ignore')

Para el 'nivel educativo' se aclaran no solo el nivel en curso (Secundario, Terciario, Primario) sino que tambien los estadios educacionales (incompleto, en curso, completo).

Por lo tanto, se consideró que el agrupamiento de niveles detallados a rangos amplios (Primaria, Secundaria, Superior) es la mejor manera de reducir la cardinalidad (de 14 a 4) y crear un feature más estable que mida el nivel socioeconómico y de recursos.

In [ ]:
# --- 2. INGENIERÍA DE NIVEL EDUCATIVO (NE) ---
col_ne = 'nivel_educativo'
mapping_ne = {
    # Primario
    'primario incompleto': 'PRIMARIO', 'primario completo': 'PRIMARIO',
    'sin estudios/ sabe leer y escribir': 'OTROS', 'escuela especial completa': 'OTROS',
    # Secundario
    'secundario incompleto': 'SECUNDARIO', 'secundario completo': 'SECUNDARIO', 'secundario en curso': 'SECUNDARIO',
    # Superior
    'terciario incompleto': 'SUPERIOR', 'terciario completo': 'SUPERIOR', 'terciario en curso': 'SUPERIOR',
    'universitario incompleto': 'SUPERIOR', 'universitario completo': 'SUPERIOR', 'universitario en curso': 'SUPERIOR',
    'desconocido': 'DESCONOCIDO'
}
df['nivel_educativo_grouped'] = df[col_ne].replace(mapping_ne)
df = pd.get_dummies(df, columns=['nivel_educativo_grouped'], prefix='NE', drop_first=False)
df = df.drop(columns=['nivel_educativo'], errors='ignore')

La variable 'situacion laboral' tiene muchas categorías de baja frecuencia.

El riesgo debe estar centrado en la inestabilidad económica o la dependencia de la persona que consulta.

Se decide  agrupar por Tipo de Ingreso/Dependencia en $\approx 4$ categorías clave:
* DESEMPLEO (incluye 'no trabaja', 'ama de casa', 'desocupade')
* FORMAL
* INFORMAL (incluye 'cuenta propista', 'trabajo informal')
* JUB/PEN (incluye 'jubilade', 'pensionada').

In [ ]:
# --- 3. INGENIERÍA DE SITUACIÓN LABORAL (SL) ---
col_sl = 'situacion_laboral'
mapping_sl = {
    'no trabaja': 'DESEMPLEO', 'desocupade': 'DESEMPLEO', 'ama de casa': 'DESEMPLEO', # Desempleo
    'trabajo formal': 'FORMAL',
    # Informal/Cuenta Propista
    'cuenta propista no registrado': 'INFORMAL', 'cuenta propista registrado': 'INFORMAL',
    'trabajo informal': 'INFORMAL', 'esporádico informal': 'INFORMAL', 'esporádico formal': 'INFORMAL',
    # Jubilación/Pensión
    'jubilade': 'JUB_PEN', 'pensionada': 'JUB_PEN',
    'desconocido': 'DESCONOCIDO'
}
df['situacion_laboral_grouped'] = df[col_sl].replace(mapping_sl)
df = pd.get_dummies(df, columns=['situacion_laboral_grouped'], prefix='SL', drop_first=False)
df = df.drop(columns=['situacion_laboral'], errors='ignore')

Para la variable 'percibe_prestacion_estatal' la pregunta clave de riesgo es: ¿Existe un soporte formal del Estado (AUH/SUAF/Pensión)?

Esto permitió agrupar en 3 categorías funcionales para:
* BENEFICIO_CLAVE (AUH/SUAF) --> SI
* NO_PERCIBE --> NO
* OTROS_BENEFICIOS (pensiones, etc.) --> SI

Se realiza un agrupamiento binario para responder a la pregunta con Si = 1 / NO = 0.

In [ ]:
# --- 4. INGENIERÍA DE PERCEPCIÓN ESTATAL (PPE) ---
col_ppe = 'percibe_prestacion_estatal'
df['prestacion_grouped'] = np.select(
    [
        df[col_ppe].str.contains('auh') | df[col_ppe].str.contains('suaf'),
        df[col_ppe].str.contains('no percibe'),
        df[col_ppe].str.contains('desconocido')
    ],
    ['BENEFICIO_CLAVE', 'NO_PERCIBE', 'DESCONOCIDO'],
    default='OTROS_BENEFICIOS' # Incluye 'otra', pensiones, alimentar
)
df = pd.get_dummies(df, columns=['prestacion_grouped'], prefix='PPE', drop_first=False)
df = df.drop(columns=[col_ppe], errors='ignore')

Para poder agrupar la variable 'vivienda', se identificó que el factor de riesgo más importante es la dependencia o inestabilidad del hogar.

Se agrupó en 4 categorías:
* PROPIA (máxima estabilidad)
* ALQUILADA (costo alto)
* PRESTADA/COMPARTIDA (dependencia)
* RIESGO/OTROS (situación de calle, ocupada).

In [ ]:
# --- 5. INGENIERÍA DE VIVIENDA (VIV) ---
col_viv = 'vivienda'
mapping_viv = {
    'propia': 'PROPIA', 'alquilada': 'ALQUILADA',
    'prestada': 'PRESTADA_COMPARTIDA', 'compartida': 'PRESTADA_COMPARTIDA',
    'desconocido': 'DESCONOCIDO',
    # Agrupar riesgo en RIESGO_ALTO
    'situación de calle': 'RIESGO_ALTO', 'ocupada': 'RIESGO_ALTO',
    'propiedad bien ganancial': 'RIESGO_ALTO', 'vivienda del pea': 'RIESGO_ALTO'
}
df['vivienda_grouped'] = df[col_viv].replace(mapping_viv)
df = pd.get_dummies(df, columns=['vivienda_grouped'], prefix='VIV', drop_first=False)
df = df.drop(columns=[col_viv], errors='ignore')

Para la variable 'obra social' la pregunta clave es: ¿Existe?

Esto permite realizar una binarización con un mapeo  lógico: $1$ debe ser la presencia del atributo --> $1 = \text{SI}$ y $0 = \text{NO/DESCONOCIDO}$.

In [ ]:
# --- 6. INGENIERÍA DE OBRA SOCIAL (OS) ---
col_os = 'obra_social'
# 6.1. Invertir el mapeo: 1 = SI, 0 = NO/DESCONOCIDO
df['obra_social_bin'] = np.where(
    df[col_os] == 'si',
    1, # SI tiene obra social
    0  # NO tiene o es desconocido
)

In [ ]:
# Eliminar la columna original
df = df.drop(columns=[col_os], errors='ignore')

In [ ]:
# --- VERIFICACIÓN DE LA INGENIERÍA ---
print("--- Verificación de la Ingeniería de Características Socioeconómicas ---")
print(f"Total de columnas después del OHE: {df.shape[1]}")

# Conteo de Obra Social
print("\nConteo de Obra Social (Binario, 1=SI):")
print(df['obra_social_bin'].value_counts().to_frame(name='Conteo').to_markdown())

# Conteo de Nivel Educativo
ne_cols = [col for col in df.columns if col.startswith('NE_')]
print("\nConteo de Nivel Educativo Agrupado (NE):")
print(df[ne_cols].sum().to_frame(name='Conteo').to_markdown())

# 3. Conteo de Situación Laboral
sl_cols = [col for col in df.columns if col.startswith('SL_')]
print("\nConteo de Situación Laboral Agrupada (SL):")
print(df[sl_cols].sum().to_frame(name='Conteo').to_markdown())

# Conteo de Estado Civil
ec_cols = [col for col in df.columns if col.startswith('EC_')]
print("\nConteo de Estado Civil Agrupado (EC):")
print(df[ec_cols].sum().to_frame(name='Conteo').to_markdown())

# Conteo de Vivienda
ec_cols = [col for col in df.columns if col.startswith('VIV_')]
print("\nConteo de Vivienda Agrupado (VIV):")
print(df[ec_cols].sum().to_frame(name='Conteo').to_markdown())

# Conteo de Percepción Estatal
ec_cols = [col for col in df.columns if col.startswith('PPE_')]
print("\nConteo de Percepción Estatal Agrupado (PPE):")
print(df[ec_cols].sum().to_frame(name='Conteo').to_markdown())

--- Verificación de la Ingeniería de Características Socioeconómicas ---
Total de columnas después del OHE: 66

Conteo de Obra Social (Binario, 1=SI):
|   obra_social_bin |   Conteo |
|------------------:|---------:|
|                 0 |      174 |
|                 1 |       61 |

Conteo de Nivel Educativo Agrupado (NE):
|                |   Conteo |
|:---------------|---------:|
| NE_DESCONOCIDO |       30 |
| NE_OTROS       |        5 |
| NE_PRIMARIO    |       29 |
| NE_SECUNDARIO  |      112 |
| NE_SUPERIOR    |       59 |

Conteo de Situación Laboral Agrupada (SL):
|                |   Conteo |
|:---------------|---------:|
| SL_DESCONOCIDO |       48 |
| SL_DESEMPLEO   |       69 |
| SL_FORMAL      |       38 |
| SL_INFORMAL    |       70 |
| SL_JUB_PEN     |       10 |

Conteo de Estado Civil Agrupado (EC):
|                        |   Conteo |
|:-----------------------|---------:|
| EC_casada              |       63 |
| EC_desconocido         |       25 |
| EC_separada_divorc

### Análisis de descriptores sobre características Clínicas, de Riesgo y Familiares:

Criterios y decisiones:

1. diagnostico / tratamiento / posee_cud --> El alto volumen de 'desconocido' ($51\%$ a $60\%$) se agrupa en 'NO' por instrucción directa.

Para diagnostico, las categorías minoritarias (consum antidep., enfermedad crónica) representan la existencia de una condición clínica. Se agrupan en 'si' para que el feature mida la presencia de cualquier riesgo clínico formal.

2. hijos_pea / convivencia_pea --> Son features limpios y binarios (si/no) que solo necesitan la conversión al formato numérico final.

In [ ]:
# Variables a codificar
cols_to_engineer = ['diagnostico', 'tratamiento', 'posee_cud', 'hijos_pea', 'convivencia_pea']

# --- 1. INGENIERÍA Y CONSOLIDACIÓN DE 'diagnostico' ---
col_diag = 'diagnostico'
# 1.1. Agrupar minorías clínicas en 'si'
df[col_diag] = df[col_diag].replace({
    'consum antidep.': 'si',
    'enfermedad crónica': 'si'
})
# 1.2. Aplicar Regla de Negocio: 'desconocido' a 'no'
df[col_diag] = df[col_diag].replace({'desconocido': 'no'})
# 1.3. Codificación Binaria (si=1, no=0)
df[f'{col_diag}_bin'] = df[col_diag].replace({'si': 1, 'no': 0})


# --- 2. INGENIERÍA DE TRATAMIENTO Y POSEE CUD ---
for col in ['tratamiento', 'posee_cud']:
    # 2.1. Aplicar Regla de Negocio: 'desconocido' a 'no'
    df[col] = df[col].replace({'desconocido': 'no'})
    # 2.2. Codificación Binaria (si=1, no=0)
    df[f'{col}_bin'] = df[col].replace({'si': 1, 'no': 0})


# --- 3. INGENIERÍA DE VARIABLES FAMILIARES ---
for col in ['hijos_pea', 'convivencia_pea']:
    # Conversión directa (ya que la limpieza dejó solo 'si'/'no')
    df[f'{col}_bin'] = df[col].replace({'si': 1, 'no': 0})


# --- 4. VERIFICACIÓN DE LA INGENIERÍA ---
print("--- Verificación Final de la Ingeniería (Clínicas y Riesgo) ---")

# Verificación de Diagnóstico (DIAG)
print("\nConteo de Diagnóstico Binario (DIAG_bin: 1=SI):")
print(df['diagnostico_bin'].value_counts().to_frame(name='Conteo').to_markdown())

# Verificación de Posesión de CUD (CUD)
print("\nConteo de Posesión de CUD Binario (posee_cud_bin: 1=SI):")
print(df['posee_cud_bin'].value_counts().to_frame(name='Conteo').to_markdown())

# Verificación de Hijos PEA
print("\nConteo de Hijos PEA Binario (hijos_pea_bin: 1=SI):")
print(df['hijos_pea_bin'].value_counts().to_frame(name='Conteo').to_markdown())

--- Verificación Final de la Ingeniería (Clínicas y Riesgo) ---

Conteo de Diagnóstico Binario (DIAG_bin: 1=SI):
|   diagnostico_bin |   Conteo |
|------------------:|---------:|
|                 0 |      193 |
|                 1 |       42 |

Conteo de Posesión de CUD Binario (posee_cud_bin: 1=SI):
|   posee_cud_bin |   Conteo |
|----------------:|---------:|
|               0 |      219 |
|               1 |       16 |

Conteo de Hijos PEA Binario (hijos_pea_bin: 1=SI):
|   hijos_pea_bin |   Conteo |
|----------------:|---------:|
|               0 |      131 |
|               1 |      104 |


/tmp/ipython-input-431716.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[f'{col_diag}_bin'] = df[col_diag].replace({'si': 1, 'no': 0})
/tmp/ipython-input-431716.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[f'{col}_bin'] = df[col].replace({'si': 1, 'no': 0})
/tmp/ipython-input-431716.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd

In [ ]:
# Verificación de Convivencia PEA
print("\nConteo de Convivencia PEA Binario (convivencia_pea_bin: 1=SI):")
print(df['convivencia_pea_bin'].value_counts().to_frame(name='Conteo').to_markdown())


Conteo de Convivencia PEA Binario (convivencia_pea_bin: 1=SI):
|   convivencia_pea_bin |   Conteo |
|----------------------:|---------:|
|                     0 |      218 |
|                     1 |       17 |


In [ ]:
# Verificación de Tratamiento (TRA)
print("\nConteo de Tratamiento Binario (TRA_bin: 1=SI):")
print(df['tratamiento_bin'].value_counts().to_frame(name='Conteo').to_markdown())


Conteo de Tratamiento Binario (TRA_bin: 1=SI):
|   tratamiento_bin |   Conteo |
|------------------:|---------:|
|                 0 |      202 |
|                 1 |       33 |


In [ ]:
#Eliminar columnas originales
df = df.drop(columns=['diagnostico', 'tratamiento', 'posee_cud', 'hijos_pea', 'convivencia_pea'], errors='ignore')

### Análisis de descriptores de violencia

La variable cantidad de personas a cargo es de conteo, pero las frecuencias de valores más altos ($5, 6, 7$) son extremadamente bajas ($<2\%$).

El agrupamiento estabiliza el modelo y mejora la interpretabilidad al agrupar casos en rangos de riesgo funcional.

* CERO (0) --> ausencia de riesgo familiar
* BAJO_RIEGO (1,2)
* MEDIO_RIESGO (3,4)
* ALTO_RIESGO ($\geq 5$)

In [ ]:
# Lógica de agrupamiento

col_count = 'cant_personas_a_cargo'
col_grouped = 'personas_cargo_grouped'

# --- 1. PASO DE LIMPIEZA INICIAL (Necesario antes del agrupamiento) ---
# Imputación Nulo/NO -> 0 (Regla de Negocio)
df[col_count] = df[col_count].replace('NO', '0')
df[col_count] = pd.to_numeric(df[col_count], errors='coerce').fillna(0).astype(int)

# --- 2. APLICACIÓN DEL AGRUPAMIENTO FUNCIONAL ---
df[col_grouped] = np.select(
    [
        df[col_count] == 0,
        df[col_count].isin([1, 2]),
        df[col_count].isin([3, 4]),
        df[col_count] >= 5
    ],
    [
        'CERO',
        'BAJO_RIESGO',
        'MEDIO_RIESGO',
        'ALTO_RIESGO'
    ],
    default='ERROR_LOGICO' # Categoría de seguridad
)

# --- 3. VERIFICACIÓN ---
verification_counts = df[col_grouped].value_counts().to_frame(name='Conteo')
total_registros = len(df)

print("--- Verificación de la Ingeniería de 'cant_personas_a_cargo' (Agrupamiento) ---")
print(f"Total de Registros: {total_registros}")
print("\nConteo por Categoría Funcional:")
print(verification_counts.to_markdown())

--- Verificación de la Ingeniería de 'cant_personas_a_cargo' (Agrupamiento) ---
Total de Registros: 235

Conteo por Categoría Funcional:
| personas_cargo_grouped   |   Conteo |
|:-------------------------|---------:|
| CERO                     |       93 |
| BAJO_RIESGO              |       89 |
| MEDIO_RIESGO             |       49 |
| ALTO_RIESGO              |        4 |


In [ ]:
# Eliminar la columna original (ya transformada)
df = df.drop(columns=[col_count], errors='ignore')

La variable modalidad_de_violencia tiene un sesgo extremo ($80\%$ es 'doméstica'), siendo 'doméstica' un valor casi constante. Se decide eliminarla por tener un valor predictivo muy bajo para la tarea de clasificación.

In [ ]:
col_modalidad = 'modalidad_de_violencia'

# Eliminación estratégica
df = df.drop(columns=[col_modalidad], errors='ignore')

print(f"Columna '{col_modalidad}' eliminada por baja variabilidad (80% es 'doméstica').")

Columna 'modalidad_de_violencia' eliminada por baja variabilidad (80% es 'doméstica').


Respecto a los tipos de violencia, la acumulación de riesgo es un fuerte predictor de desenlaces graves en violencia de género.

La decisión de crear la variable SUMA_VIOLENCIA, puede resultar más útil que analizar cada tipo por separado, ya que captura la complejidad del caso(cuántos tipos de violencia se acumulan).

In [ ]:
# Lista de las 8 variables de violencia (asumiendo que están en formato numérico 0/1)

violence_cols = [
    'fisica', 'psicologica', 'sexual', 'economica',
    'simbolica', 'ambiental', 'politica', 'digital'
]
original_count_col = 'cant_tipos_violencias_por_persona'

# --- 1. CREACIÓN DEL FEATURE DE SUMA (Acumulación de Riesgo) ---

# Aseguramos que las 8 flags sean enteras para la suma
for col in violence_cols:
    # Este paso finaliza la Ingeniería de las 8 flags a tipo int64
    df[col] = df[col].astype(int)

# Crear el feature de suma
df['SUMA_VIOLENCIA'] = df[violence_cols].sum(axis=1).astype(int)

# 3. Verificación de la Ingeniería
print("--- Verificación de la Ingeniería de Características (Violencia) ---")

print("\n--- Conteo del Nuevo Feature 'SUMA_VIOLENCIA' ---")
print(df['SUMA_VIOLENCIA'].value_counts().sort_index().to_frame(name='Conteo').to_markdown())

print(f"\nTipo de dato de 'SUMA_VIOLENCIA': {df['SUMA_VIOLENCIA'].dtype}")

--- Verificación de la Ingeniería de Características (Violencia) ---

--- Conteo del Nuevo Feature 'SUMA_VIOLENCIA' ---
|   SUMA_VIOLENCIA |   Conteo |
|-----------------:|---------:|
|                0 |       60 |
|                1 |       40 |
|                2 |       51 |
|                3 |       36 |
|                4 |       28 |
|                5 |       13 |
|                6 |        6 |
|                7 |        1 |

Tipo de dato de 'SUMA_VIOLENCIA': int64


In [ ]:
# Eliminar la columna original mal registrada
df = df.drop(columns=[original_count_col], errors='ignore')

print(f"La columna '{original_count_col}' ha sido eliminada.")

La columna 'cant_tipos_violencias_por_persona' ha sido eliminada.


### Análisis de variables sociales

Para la variable personas a cargo, un agrupamiento funcional de no + otros_as $\rightarrow$ NO_OTROS, permite consolidar la ausencia de hijos/as a cargo con los casos de muy baja frecuencia.

En cuanto a red vincular, se crea un agrupamiento de no + vecinas_os $\rightarrow$ RV_NO_APOYO, que representa la ausencia de una red de apoyo principal.

In [ ]:
# --- 1. INGENIERÍA DE 'personas_a_cargo' (PER) ---

col_pc = 'personas_a_cargo'
new_col_pc = 'personas_cargo_grouped'

# 1.1. Agrupamiento Funcional
# Justificación: Consolidar las categorías de baja frecuencia ('otros_as') y la ausencia ('no')
# en una categoría funcional para el modelo ('NO_OTROS').
df[new_col_pc] = df[col_pc].replace({
    'no': 'NO_OTROS',
    'otros_as': 'NO_OTROS'
})
# Las categorías 'hija_hijo' y 'desconocido' se mantienen.

# 1.2. Aplicar OHE
df = pd.get_dummies(df, columns=[new_col_pc], prefix='PER', drop_first=False)


# --- 2. INGENIERÍA DE 'red_vincular' (RV) ---

col_rv = 'red_vincular'
new_col_rv = 'red_vincular_grouped'

# 2.1. Renombrar y Agrupar
# Justificación: Agrupar la categoría de baja frecuencia ('vecinas_os') con la ausencia ('no') en 'NO_OTROS'.
df[new_col_rv] = df[col_rv].replace({
    'no': 'NO_OTROS',
    'vecinas_os': 'NO_OTROS',
    'amigas_os': 'AMIGOS' # Renombrar a AMIGOS por claridad
})

# 2.2. Aplicar OHE
df = pd.get_dummies(df, columns=[new_col_rv], prefix='RV', drop_first=False)


# --- 4. VERIFICACIÓN DE LA INGENIERÍA ---
print("--- Verificación de la Ingeniería de Características ---")

# Conteo de Personas a Cargo
pc_cols = [col for col in df.columns if col.startswith('PER_')]
print("\nConteo de Personas a Cargo Agrupadas (PER):")
print(df[pc_cols].sum().to_frame(name='Conteo').to_markdown())

# Conteo de Red Vincular
rv_cols = [col for col in df.columns if col.startswith('RV_')]
print("\nConteo de Red Vincular Agrupada (RV):")
print(df[rv_cols].sum().to_frame(name='Conteo').to_markdown())

--- Verificación de la Ingeniería de Características ---

Conteo de Personas a Cargo Agrupadas (PER):
|                 |   Conteo |
|:----------------|---------:|
| PER_NO_OTROS    |       32 |
| PER_desconocido |       52 |
| PER_hija_hijo   |      151 |

Conteo de Red Vincular Agrupada (RV):
|                              |   Conteo |
|:-----------------------------|---------:|
| RV_AMIGOS                    |       61 |
| RV_NO_OTROS                  |       12 |
| RV_desconocido               |       62 |
| RV_parientes_convivientes    |       45 |
| RV_parientes_no_convivientes |       55 |


In [ ]:
# ELIMINACIÓN DE COLUMNAS ORIGINALES
df = df.drop(columns=[col_pc, col_rv], errors='ignore')

### Análisis de variable objetivo y variables identificadoras de denuncia:

Las variables medidas_de_proteccion y fecha_fin_de_vigencia se utilizaron para definir la "verdad" de la columna denuncio. Incluirlas como predictores causaría que el modelo las use como atajos, inflando las métricas de forma irreal.

Decisión: Eliminarlas para evitar la fuga de datos (Data Leakage).

Por último, se binariza 'denuncio' como 'denuncio_target' y se convierte  el target numérico del modelo:
* 'NO' $\rightarrow 1$ (Alerta)
* 'SI' $\rightarrow 0$ (Base)

In [ ]:
# INGENIERÍA DE FEATURES (Eliminación por Data Leakage) ---

col_medida = 'medidas_de_proteccion'
col_fecha = 'fecha_fin_de_vigencia'

# Eliminar las columnas que se usaron en la Regla de Oro (medidas y fecha)
df = df.drop(columns=[col_medida, col_fecha], errors='ignore')

In [ ]:
# Verificar la eliminación de las columnas de fuga de datos
if col_medida not in df.columns and col_fecha not in df.columns:
    print("\n Las variables de fuga de datos (medidas y fecha) fueron eliminadas con éxito.")
else:
    print("\n Alerta: Las variables de fuga de datos aún existen. Revisar la limpieza previa.")


 Las variables de fuga de datos (medidas y fecha) fueron eliminadas con éxito.


In [ ]:
# INGENIERÍA DEL TARGET (Binarización) ---

col_denuncio = 'denuncio'

# Crear la variable target numérica 'denuncio_target'
# Coherente con la Alerta Temprana: NO Denuncia = 1 (La clase de interés/riesgo)
df['denuncio_target'] = df[col_denuncio].replace({'no': 1, 'si': 0})

/tmp/ipython-input-2296897140.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['denuncio_target'] = df[col_denuncio].replace({'no': 1, 'si': 0})


In [ ]:
# Eliminar la columna de texto original
df = df.drop(columns=[col_denuncio], errors='ignore')

In [ ]:
# VERIFICACIÓN

print("--- Verificación Final de la Ingeniería de Características ---")
print(f"Total de columnas restantes: {df.shape[1]}")

# Verificar el target numérico
print("\nConteo Final del Target Numérico ('denuncio_target'):")
print(df['denuncio_target'].value_counts().to_frame(name='Conteo').to_markdown())

--- Verificación Final de la Ingeniería de Características ---
Total de columnas restantes: 68

Conteo Final del Target Numérico ('denuncio_target'):
|   denuncio_target |   Conteo |
|------------------:|---------:|
|                 0 |      140 |
|                 1 |       95 |


### Conversion de variables boleanas a binarias

In [ ]:
# 1. Seleccionar todas las columnas booleanas (bool)
boolean_cols = df.select_dtypes(include=['bool']).columns

# 2. Aplicar la conversión a entero (int)
# True se convierte automáticamente a 1, y False a 0.
df[boolean_cols] = df[boolean_cols].astype(int)

### Terminada la primera etapa, revisamos nuevamente el df:

In [ ]:
df.head(10)

,fisica,psicologica,sexual,economica,simbolica,ambiental,politica,digital,INGR_CANAL_ASISTENCIAL,INGR_CANAL_ESPONTANEO,...,SUMA_VIOLENCIA,PER_NO_OTROS,PER_desconocido,PER_hija_hijo,RV_AMIGOS,RV_NO_OTROS,RV_desconocido,RV_parientes_convivientes,RV_parientes_no_convivientes,denuncio_target
0,1,1,0,0,0,0,0,0,0,0,...,2,1,0,0,0,0,0,1,0,1
1,1,1,0,1,0,1,0,0,0,0,...,4,0,0,1,0,0,0,0,1,0
2,1,1,0,0,0,0,0,0,0,0,...,2,1,0,0,0,0,0,0,1,1
3,1,1,0,0,0,0,0,0,0,0,...,2,0,1,0,0,0,0,0,1,1
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
6,0,1,0,1,0,0,0,0,0,0,...,2,0,0,1,0,0,0,0,1,0
7,1,0,0,1,0,0,0,0,0,0,...,2,0,0,1,0,0,0,0,1,0
8,1,1,1,0,1,0,0,0,0,0,...,4,0,0,1,0,0,0,0,1,0
9,1,1,1,1,0,0,0,0,0,0,...,4,0,0,1,0,0,0,0,1,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   fisica                        235 non-null    int64
 1   psicologica                   235 non-null    int64
 2   sexual                        235 non-null    int64
 3   economica                     235 non-null    int64
 4   simbolica                     235 non-null    int64
 5   ambiental                     235 non-null    int64
 6   politica                      235 non-null    int64
 7   digital                       235 non-null    int64
 8   INGR_CANAL_ASISTENCIAL        235 non-null    int64
 9   INGR_CANAL_ESPONTANEO         235 non-null    int64
 10  INGR_CANAL_JUDICIAL           235 non-null    int64
 11  INGR_CANAL_TERCERO            235 non-null    int64
 12  INGR_DESCONOCIDO              235 non-null    int64
 13  NAC_ARGENTINA                 235 n

# Descarga de CSV limpio

In [ ]:
# 1. Guardar el DataFrame a un archivo CSV.
# index=False evita que pandas guarde la columna de índice numérico como una columna extra.
df.to_csv('dataset_transformado_ok.csv', index=False)

print("Archivo 'dataset_transformado_ok.csv' generado con éxito en tu entorno de Colab.")

# 2. Descargar el archivo
# La siguiente línea inicia la descarga a tu computadora local:
from google.colab import files
files.download('dataset_transformado_ok.csv')

print("Descarga iniciada a tu computadora.")

Archivo 'dataset_transformado_ok.csv' generado con éxito en tu entorno de Colab.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Descarga iniciada a tu computadora.
